# Investigating Az glitches
Craig Lage - 25-Nov-24

In [ ]:
# Directory to store the data
from pathlib import Path
data_dir = Path("./plots")
data_dir.mkdir(exist_ok=True, parents=True)

# Choose the event
dayObs = 20241121
seqNum = 221

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.ticker import FuncFormatter
import numpy as np
import pandas as pd
from astropy.time import Time
from lsst.summit.utils.tmaUtils import TMAEventMaker, getEfdData, \
plotEvent, getAzimuthElevationDataForEvent, getM1M3HardpointDataForEvent
from lsst.summit.utils.efdUtils import makeEfdClient
from lsst_efd_client.efd_helper import merge_packed_time_series
from lsst.ts.observing.utilities.decorated_logger import DecoratedLogger
logger = DecoratedLogger.get_decorated_logger()

In [ ]:
def tickFormatter(value: float, tick_number: float) -> str:
    # Convert the value to a string without subtracting large numbers
    # tick_number is unused.
    return f"{value:.2f}"


In [ ]:
client = makeEfdClient()
eventMaker = TMAEventMaker()

# First, RubinTV TMA event plot

In [ ]:
event = eventMaker.getEvent(dayObs, seqNum)
fig = plt.figure(figsize=(10, 8))
x = plotEvent(client, event, fig, doFilterResiduals=True)
plt.savefig(str(data_dir / f"Az_Glitch_Plot1_{dayObs}_{seqNum}"))

# TMA Mount data

In [ ]:
azimuthData, elevationData = \
        getAzimuthElevationDataForEvent(client, event=event)

figure = plt.figure(figsize=(10, 8))
plt.suptitle(f"TMA plots {dayObs} - {seqNum}", fontsize = 18)
[ax1, ax2, ax3, ax4] = figure.subplots(4,1,sharex=True, \
           gridspec_kw={"hspace": 0, "height_ratios": [1,1,1,1]})
# Use the native color cycle for the lines. Because they're on different
# axes they don't cycle by themselves
lineColors = [p["color"] for p in plt.rcParams["axes.prop_cycle"]]
nColors = len(lineColors)
colorCounter = 0

ax1.plot(azimuthData["actualPosition"], label="Az Pos", c=lineColors[colorCounter % nColors])
colorCounter += 1
ax1.yaxis.set_major_formatter(FuncFormatter(tickFormatter))
ax1.set_ylabel("Az Pos(degrees)")

ax1_twin = ax1.twinx()
ax1_twin.plot(
    elevationData["actualPosition"], label="El Pos", c=lineColors[colorCounter % nColors]
)
colorCounter += 1
ax1_twin.set_ylabel("El Pos(degrees)")
ax1.legend(loc='upper left')
ax1_twin.legend(loc='upper right')

ax2.plot(azimuthData["actualVelocity"], label="Az Vel", c=lineColors[colorCounter % nColors])
colorCounter += 1
ax2.yaxis.set_major_formatter(FuncFormatter(tickFormatter))
ax2.set_ylabel("Az Vel(degrees/sec)")

ax2_twin = ax2.twinx()
ax2_twin.plot(
    elevationData["actualVelocity"], label="El Vel", c=lineColors[colorCounter % nColors]
)
colorCounter += 1
ax2_twin.set_ylabel("El Vel(degrees/sec)")
ax2.legend(loc='upper left')
ax2_twin.legend(loc='upper right')


ax3.plot(azimuthData["actualAcceleration"], label="Az Acc", c=lineColors[colorCounter % nColors])
colorCounter += 1
ax3.yaxis.set_major_formatter(FuncFormatter(tickFormatter))
ax3.set_ylabel("Az acc(degrees/sec^2)")

ax3_twin = ax3.twinx()
ax3_twin.plot(
    elevationData["actualAcceleration"], label="El Acc", c=lineColors[colorCounter % nColors]
)
colorCounter += 1
ax3_twin.set_ylabel("El acc(degrees/sec^2)")
ax3.legend(loc='upper left')
ax3_twin.legend(loc='upper right')


ax4_twin = ax4.twinx()
ax4.plot(azimuthData["actualTorque"], label="Az torque", c=lineColors[colorCounter % nColors])
colorCounter += 1
ax4_twin.plot(elevationData["actualTorque"], label="El torque", c=lineColors[colorCounter % nColors])
colorCounter += 1
ax4.set_ylabel("Az torque (Nm)")
ax4_twin.set_ylabel("El torque (Nm)")
ax4.set_xlabel("Time (UTC)")  # yes, it really is UTC, matplotlib converts this automatically!
ax4.legend(loc='upper left')
ax4_twin.legend(loc='upper right')

# put the ticks at an angle, and right align with the tick marks
ax4.set_xticks(ax4.get_xticks())  # needed to supress a user warning
xlabels = ax4.get_xticks()
ax4.set_xticklabels(xlabels)
ax4.tick_params(axis='x', rotation=45)
ax4.xaxis.set_major_locator(mdates.AutoDateLocator())
ax4.xaxis.set_major_formatter(mdates.DateFormatter("%H:%M:%S"))

"""
deltaT = 225.0 / 86400.0
lengthT = 1.0 / 86400.0
xlim = ax1.get_xlim()
new_xlim = (xlim[0] + deltaT, xlim[0] + deltaT + lengthT)
ax1.set_xlim(new_xlim)
ax2.set_xlim(new_xlim)
ax3.set_xlim(new_xlim)
ax4.set_xlim(new_xlim)
"""

plt.savefig(str(data_dir / f"Az_Glitch_Plot2_{dayObs}_{seqNum}"))

# Next Az cable wrap plots

In [ ]:
wrapData = getEfdData(
        client,
        "lsst.sal.MTMount.azimuthCableWrap",
        event=event,
    )
figure = plt.figure(figsize=(10, 4))
plt.suptitle(f"Azimuth Cable Wrap plots {dayObs} - {seqNum}", fontsize = 18)
[ax1, ax2] = figure.subplots(2,1,sharex=True, \
           gridspec_kw={"hspace": 0, "height_ratios": [1,1]})
# Use the native color cycle for the lines. Because they're on different
# axes they don't cycle by themselves
lineColors = [p["color"] for p in plt.rcParams["axes.prop_cycle"]]
nColors = len(lineColors)
colorCounter = 0

ax1.plot(wrapData["actualPositionDrive0"], label="Pos Drive0", c=lineColors[colorCounter % nColors])
colorCounter += 1
ax1.yaxis.set_major_formatter(FuncFormatter(tickFormatter))
ax1.set_ylabel("Pos Drive0(degrees)")

ax1_twin = ax1.twinx()
ax1_twin.plot(
    wrapData["actualPositionDrive1"], label="Pos Drive1", ls = '--', c=lineColors[colorCounter % nColors]
)
colorCounter += 1
ax1_twin.set_ylabel("Pos Drive 1(degrees)")
ax1.legend(loc='upper left')
ax1_twin.legend(loc='upper right')


ax2.plot(wrapData["actualVelocityDrive0"], label="Vel Drive0", c=lineColors[colorCounter % nColors])
colorCounter += 1
ax2.yaxis.set_major_formatter(FuncFormatter(tickFormatter))
ax2.set_ylabel("Vel Drive0(degrees/sec)")

ax2_twin = ax2.twinx()
ax2_twin.plot(
    wrapData["actualVelocityDrive1"], label="Vel Drive1", ls = '--', c=lineColors[colorCounter % nColors]
)
colorCounter += 1
ax2_twin.set_ylabel("Vel Drive 1(degrees)")
ax2.legend(loc='upper left')
ax2_twin.legend(loc='upper right')

# put the ticks at an angle, and right align with the tick marks
ax2.set_xticks(ax2.get_xticks())  # needed to supress a user warning
xlabels = ax2.get_xticks()
ax2.set_xticklabels(xlabels)
ax2.tick_params(axis='x', rotation=45)
ax2.xaxis.set_major_locator(mdates.AutoDateLocator())
ax2.xaxis.set_major_formatter(mdates.DateFormatter("%H:%M:%S"))

plt.savefig(str(data_dir / f"Az_Glitch_Plot3_{dayObs}_{seqNum}"))

# Gyro and Accelerometer data

In [ ]:
gyroData = getEfdData(
        client,
        "lsst.sal.MTM1M3.gyroData",
        event=event,
    )
accelData = getEfdData(
        client,
        "lsst.sal.MTM1M3.accelerometerData",
        event=event,
    )
figure = plt.figure(figsize=(10, 4))
plt.suptitle(f"Azimuth Gyro and Accel plots {dayObs} - {seqNum}", fontsize = 18)
[ax1, ax2] = figure.subplots(2,1,sharex=True, \
           gridspec_kw={"hspace": 0, "height_ratios": [1,1]})
# Use the native color cycle for the lines. Because they're on different
# axes they don't cycle by themselves
lineColors = [p["color"] for p in plt.rcParams["axes.prop_cycle"]]
nColors = len(lineColors)
colorCounter = 0

ax1.plot(gyroData["angularVelocityX"], label="Ang Vel X", 
         c=lineColors[colorCounter % nColors])
colorCounter += 1
ax1.plot(gyroData["angularVelocityY"], label="Ang Vel Y", 
         c=lineColors[colorCounter % nColors])
colorCounter += 1
ax1.plot(gyroData["angularVelocityZ"], label="Ang Vel Z", 
         c=lineColors[colorCounter % nColors])
colorCounter += 1
ax1.yaxis.set_major_formatter(FuncFormatter(tickFormatter))
ax1.set_ylabel("Angular velocity (deg/sec)")

colorCounter += 1
ax1.legend(loc='upper left')

ax2.plot(accelData["angularAccelerationX"], label="Ang Acc X", 
         c=lineColors[colorCounter % nColors])
colorCounter += 1
ax2.plot(accelData["angularAccelerationY"], label="Ang Acc Y", 
         c=lineColors[colorCounter % nColors])
colorCounter += 1
ax2.plot(accelData["angularAccelerationZ"], label="Ang Acc Z", 
         c=lineColors[colorCounter % nColors])
colorCounter += 1
ax2.yaxis.set_major_formatter(FuncFormatter(tickFormatter))
ax2.set_ylabel("Angular accel (deg/sec^2)")

colorCounter += 1
ax2.legend(loc='upper right')

# put the ticks at an angle, and right align with the tick marks
ax2.set_xticks(ax2.get_xticks())  # needed to supress a user warning
xlabels = ax2.get_xticks()
ax2.set_xticklabels(xlabels)
ax2.tick_params(axis='x', rotation=45)
ax2.xaxis.set_major_locator(mdates.AutoDateLocator())
ax2.xaxis.set_major_formatter(mdates.DateFormatter("%H:%M:%S"))

"""
deltaT = 225.0 / 86400.0
lengthT = 1.0 / 86400.0
xlim = ax1.get_xlim()
new_xlim = (xlim[0] + deltaT, xlim[0] + deltaT + lengthT)
ax1.set_xlim(new_xlim)
ax2.set_xlim(new_xlim)
#ax3.set_xlim(new_xlim)
#ax4.set_xlim(new_xlim)
"""
plt.savefig(str(data_dir / f"Az_Glitch_Plot4_{dayObs}_{seqNum}"))

In [ ]:
azimuthData, elevationData = \
        getAzimuthElevationDataForEvent(client, event=event)
hardpointData = getM1M3HardpointDataForEvent(client, event=event)
accelData = getEfdData(
        client,
        "lsst.sal.MTM1M3.accelerometerData",
        event=event,
    )

figure = plt.figure(figsize=(10, 8))
plt.suptitle(f"TMA plots {dayObs} - {seqNum}", fontsize = 18)
[ax1, ax2, ax3] = figure.subplots(3,1,sharex=True, \
           gridspec_kw={"hspace": 0, "height_ratios": [1,1,1]})
# Use the native color cycle for the lines. Because they're on different
# axes they don't cycle by themselves
lineColors = [p["color"] for p in plt.rcParams["axes.prop_cycle"]]
nColors = len(lineColors)
colorCounter = 0

# Calculate Max hardpoint force
maxForce = np.nanmax(hardpointData["maxForce"].to_numpy())

for hp_index in range(6):
    ax1.plot(
        hardpointData[f"measuredForce{hp_index}"],
        label=f"HP_{hp_index}",
        c=lineColors[colorCounter % nColors],
    )
    colorCounter += 1
ax1.axhline(500.0, ls="-.", color="goldenrod")
ax1.axhline(-500.0, ls="-.", color="goldenrod")
ax1.axhline(1000.0, ls="-.", color="red")
ax1.axhline(-1000.0, ls="-.", color="red")
ax1.yaxis.set_major_formatter(FuncFormatter(tickFormatter))
ax1.set_ylabel("HP force (N)")
ax1.set_xticks([])  # remove x tick labels on the hidden upper x-axis
if maxForce > 2000.0:
    ax1.set_ylim(-4000.0, 4000.0)
    ax1.set_yticks([-2000, 0.0, 2000])
elif maxForce > 1000.0:
    ax1.set_ylim(-2000.0, 2000.0)
    ax1.set_yticks([-1000, 0.0, 1000])
else:
    ax1.set_ylim(-1000.0, 1000.0)
    ax1.set_yticks([-500, 0.0, 500])
ax1.legend(loc='upper right')

ax2.plot(azimuthData["actualTorque"], label="Az torque", 
         marker='x', c=lineColors[colorCounter % nColors])
colorCounter += 1
ax2.yaxis.set_major_formatter(FuncFormatter(tickFormatter))
ax2.set_ylabel("Az torque (N-m)")
ax2.legend(loc='upper left')

ax3.plot(accelData["angularAccelerationZ"], label="Ang Acc Z", 
         marker='x', c=lineColors[colorCounter % nColors])
colorCounter += 1
ax3.yaxis.set_major_formatter(FuncFormatter(tickFormatter))
ax3.set_ylabel("Angular accel (deg/sec^2)")
ax3.legend(loc='upper left')

# put the ticks at an angle, and right align with the tick marks
ax3.set_xticks(ax3.get_xticks())  # needed to supress a user warning
xlabels = ax3.get_xticks()
ax3.set_xticklabels(xlabels)
ax3.tick_params(axis='x', rotation=45)
ax3.xaxis.set_major_locator(mdates.AutoDateLocator())
ax3.xaxis.set_major_formatter(mdates.DateFormatter("%H:%M:%S"))

deltaT = 12.8 / 86400.0
lengthT = 1.0 / 86400.0
xlim = ax1.get_xlim()
new_xlim = (xlim[0] + deltaT, xlim[0] + deltaT + lengthT)
ax1.set_xlim(new_xlim)
ax2.set_xlim(new_xlim)
ax3.set_xlim(new_xlim)

plt.savefig(str(data_dir / f"Az_Glitch_Plot5_{dayObs}_{seqNum}"))


In [ ]:
azimuthData, elevationData = \
        getAzimuthElevationDataForEvent(client, event=event)
hardpointData = getM1M3HardpointDataForEvent(client, event=event)
accelData = getEfdData(
        client,
        "lsst.sal.MTM1M3.accelerometerData",
        event=event,
    )

packed_dataframe = await client.select_time_series(
    "lsst.sal.ESS.accelerometer", ["*"], event.begin, event.end
)

figure = plt.figure(figsize=(12, 8))
plt.suptitle(f"TMA plots {dayObs} - {seqNum}", fontsize = 18)
[ax1, ax2, ax3, ax4, ax5, ax6] = figure.subplots(6,1,sharex=True, \
           gridspec_kw={"hspace": 0, "height_ratios": [1,1,1,1,1,1]})
# Use the native color cycle for the lines. Because they're on different
# axes they don't cycle by themselves
lineColors = [p["color"] for p in plt.rcParams["axes.prop_cycle"]]
nColors = len(lineColors)
colorCounter = 0

# Calculate Max hardpoint force
maxForce = np.nanmax(hardpointData["maxForce"].to_numpy())

for hp_index in range(6):
    ax1.plot(
        hardpointData[f"measuredForce{hp_index}"],
        label=f"HP_{hp_index}",
        c=lineColors[colorCounter % nColors],
    )
    colorCounter += 1
ax1.axhline(500.0, ls="-.", color="goldenrod")
ax1.axhline(-500.0, ls="-.", color="goldenrod")
ax1.axhline(1000.0, ls="-.", color="red")
ax1.axhline(-1000.0, ls="-.", color="red")
ax1.yaxis.set_major_formatter(FuncFormatter(tickFormatter))
ax1.set_ylabel("HP force(N)")
ax1.set_xticks([])  # remove x tick labels on the hidden upper x-axis
if maxForce > 2000.0:
    ax1.set_ylim(-4000.0, 4000.0)
    ax1.set_yticks([-2000, 0.0, 2000])
elif maxForce > 1000.0:
    ax1.set_ylim(-2000.0, 2000.0)
    ax1.set_yticks([-1000, 0.0, 1000])
else:
    ax1.set_ylim(-1000.0, 1000.0)
    ax1.set_yticks([-500, 0.0, 500])
ax1.legend(loc='upper right')

ax2.plot(azimuthData["actualTorque"], label="Az torque", 
         marker='x', c=lineColors[colorCounter % nColors])
colorCounter += 1
ax2.yaxis.set_major_formatter(FuncFormatter(tickFormatter))
ax2.set_ylabel("Az torque(N-m)")
ax2.legend(loc='upper left')

ax3.plot(accelData["angularAccelerationZ"], label="Ang Acc Z", 
         marker='x', c=lineColors[colorCounter % nColors])
colorCounter += 1
ax3.yaxis.set_major_formatter(FuncFormatter(tickFormatter))
ax3.set_ylabel("Ang acc\n(deg/sec^2)")
ax3.legend(loc='upper left')

baseField = "accelerationX"
sensorName = "SST top end ring +x -y"
sub_dataframe = packed_dataframe.loc[packed_dataframe.sensorName == sensorName]
ax4.set_ylabel("Acc(m/s^2)")

df = merge_packed_time_series(
    sub_dataframe,
    baseField,
    stride=1,
    ref_timestamp_col="timestamp",
    fmt="unix_tai",
    scale="tai",
)
df[baseField].plot(ax=ax4, label=f"{sensorName} {baseField}")
ax4.legend()

sensorName = "SST top end ring -x -y"
sub_dataframe = packed_dataframe.loc[packed_dataframe.sensorName == sensorName]
ax5.set_ylabel("Acc(m/s^2)")

df = merge_packed_time_series(
    sub_dataframe,
    baseField,
    stride=1,
    ref_timestamp_col="timestamp",
    fmt="unix_tai",
    scale="tai",
)
df[baseField].plot(ax=ax5, label=f"{sensorName} {baseField}")
ax5.legend()

sensorName = "SST spider spindle"
sub_dataframe = packed_dataframe.loc[packed_dataframe.sensorName == sensorName]
ax6.set_ylabel("Acc(m/s^2)")

df = merge_packed_time_series(
    sub_dataframe,
    baseField,
    stride=1,
    ref_timestamp_col="timestamp",
    fmt="unix_tai",
    scale="tai",
)
df[baseField].plot(ax=ax6, label=f"{sensorName} {baseField}")
ax6.legend()

# put the ticks at an angle, and right align with the tick marks
ax6.set_xticks(ax6.get_xticks())  # needed to supress a user warning
xlabels = ax6.get_xticks()
ax6.set_xticklabels(xlabels)
ax6.tick_params(axis='x', rotation=45)
ax6.xaxis.set_major_locator(mdates.AutoDateLocator())
ax6.xaxis.set_major_formatter(mdates.DateFormatter("%H:%M:%S"))


deltaT = 12.0 / 86400.0
lengthT = 5.0 / 86400.0
xlim = ax1.get_xlim()
new_xlim = (xlim[0] + deltaT, xlim[0] + deltaT + lengthT)
ax1.set_xlim(new_xlim)
ax2.set_xlim(new_xlim)
ax3.set_xlim(new_xlim)
ax4.set_xlim(new_xlim)
ax5.set_xlim(new_xlim)
ax6.set_xlim(new_xlim)

plt.savefig(str(data_dir / f"Az_Glitch_Plot6_{dayObs}_{seqNum}"))


In [ ]:
begin = Time("2024-11-26T12:33:20", scale='utc')
end = Time("2024-11-26T12:33:24", scale='utc')

azimuthData = getEfdData(
        client,
        "lsst.sal.MTMount.azimuth",
        begin=begin,
        end=end
    )

hardpointData = getEfdData(
        client,
        "lsst.sal.MTM1M3.hardpointActuatorData",
        begin=begin,
        end=end
    )

accelData = getEfdData(
        client,
        "lsst.sal.MTM1M3.accelerometerData",
        begin=begin,
        end=end
    )

packed_dataframe = await client.select_time_series(
    "lsst.sal.ESS.accelerometer", ["*"], begin, end
)

figure = plt.figure(figsize=(12, 8))
plt.suptitle(f"TMA plots 26Nov24 Earthquake", fontsize = 18)
[ax1, ax2, ax3, ax4, ax5, ax6] = figure.subplots(6,1,sharex=True, \
           gridspec_kw={"hspace": 0, "height_ratios": [1,1,1,1,1,1]})
axs = [ax1, ax2, ax3, ax4, ax5, ax6]
# Use the native color cycle for the lines. Because they're on different
# axes they don't cycle by themselves
lineColors = [p["color"] for p in plt.rcParams["axes.prop_cycle"]]
nColors = len(lineColors)
colorCounter = 0

# Calculate Max hardpoint force
maxForce = 400.0

for hp_index in range(6):
    ax1.plot(
        hardpointData[f"measuredForce{hp_index}"],
        label=f"HP_{hp_index}",
        c=lineColors[colorCounter % nColors],
    )
    colorCounter += 1
ax1.axhline(500.0, ls="-.", color="goldenrod")
ax1.axhline(-500.0, ls="-.", color="goldenrod")
ax1.axhline(1000.0, ls="-.", color="red")
ax1.axhline(-1000.0, ls="-.", color="red")
ax1.yaxis.set_major_formatter(FuncFormatter(tickFormatter))
ax1.set_ylabel("HP force(N)")
ax1.set_xticks([])  # remove x tick labels on the hidden upper x-axis
if maxForce > 2000.0:
    ax1.set_ylim(-4000.0, 4000.0)
    ax1.set_yticks([-2000, 0.0, 2000])
elif maxForce > 1000.0:
    ax1.set_ylim(-2000.0, 2000.0)
    ax1.set_yticks([-1000, 0.0, 1000])
else:
    ax1.set_ylim(-200.0, 200.0)
    ax1.set_yticks([-100, 0.0, 100])
ax1.legend(loc='upper right')


ax2.plot(azimuthData["actualAcceleration"], label="Az acc", 
         marker='x', c=lineColors[colorCounter % nColors])
colorCounter += 1
ax2.yaxis.set_major_formatter(FuncFormatter(tickFormatter))
ax2.set_ylabel("Az acc\n(deg/s^2)")
ax2.legend(loc='upper left')

ax3.plot(accelData["angularAccelerationZ"], label="Ang Acc Z", 
         marker='x', c=lineColors[colorCounter % nColors])
colorCounter += 1
ax3.yaxis.set_major_formatter(FuncFormatter(tickFormatter))
ax3.set_ylabel("Ang acc\n(deg/sec^2)")
ax3.legend(loc='upper left')


baseField = "accelerationX"
sensorName = "SST top end ring +x -y"
sub_dataframe = packed_dataframe.loc[packed_dataframe.sensorName == sensorName]
ax4.set_ylabel("Acc(m/s^2)")

df = merge_packed_time_series(
    sub_dataframe,
    baseField,
    stride=1,
    ref_timestamp_col="timestamp",
    fmt="unix_tai",
    scale="tai",
)
df[baseField].plot(ax=ax4, label=f"{sensorName} {baseField}")
ax4.legend()

sensorName = "SST top end ring -x -y"
sub_dataframe = packed_dataframe.loc[packed_dataframe.sensorName == sensorName]
ax5.set_ylabel("Acc(m/s^2)")

df = merge_packed_time_series(
    sub_dataframe,
    baseField,
    stride=1,
    ref_timestamp_col="timestamp",
    fmt="unix_tai",
    scale="tai",
)
df[baseField].plot(ax=ax5, label=f"{sensorName} {baseField}")
ax5.legend()

sensorName = "SST spider spindle"
sub_dataframe = packed_dataframe.loc[packed_dataframe.sensorName == sensorName]
ax6.set_ylabel("Acc(m/s^2)")

df = merge_packed_time_series(
    sub_dataframe,
    baseField,
    stride=1,
    ref_timestamp_col="timestamp",
    fmt="unix_tai",
    scale="tai",
)
df[baseField].plot(ax=ax6, label=f"{sensorName} {baseField}")


start_time = ax6.get_xlim()[0]
vline = start_time + 2.9 / 86400.0
for ax in axs:
    ax.axvline(vline, color='black', ls='--')
# put the ticks at an angle, and right align with the tick marks
ax6.set_xticks(ax6.get_xticks())  # needed to supress a user warning
xlabels = ax6.get_xticks()
ax6.set_xticklabels(xlabels)
ax6.tick_params(axis='x', rotation=45)
ax6.xaxis.set_major_locator(mdates.AutoDateLocator())
ax6.xaxis.set_major_formatter(mdates.DateFormatter("%H:%M:%S"))

plt.savefig(str(data_dir / f"Earthquake_Plot_26Nov24"))
